In [2]:
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

def logga(meddelande):
    tid = time.strftime("%Y-%m-%d %H:%M:%S")
    rad = f"[{tid}] {meddelande}"
    print(rad)
    with open("transactions_logs.txt", "a", encoding="utf-8") as f:
        f.write(rad + "\n")


In [3]:
logga("Loading CSV-files...")

df_customers = pd.read_csv("./data/sebank_customers_with_accounts.csv")
df_transactions = pd.read_csv("./data/transactions.csv")

logga(f"Extracted {len(df_customers)} customers and {len(df_transactions)} transactions.")

df_transactions["timestamp"] = pd.to_datetime(df_transactions["timestamp"])


[2025-06-04 17:24:14] Loading CSV-files...
[2025-06-04 17:24:14] Extracted 1000 customers and 100000 transactions.


In [4]:
logga("Starting currency converter to SEK...")

conversion_rates = {
    "DKK": 1.5527, "EUR": 11.6084, "GBP": 13.7361, "JPY": 0.0660, "NOK": 0.9830,
    "RMB": 1.4368, "USD": 10.2930, "ZAR": 0.5792, "ZMW": 0.3584, "AUD": 6.9731,
    "BGN": 5.8452, "BRL": 1.9697, "CAD": 7.7143, "CHF": 12.0045, "CNY": 1.4680,
    "CZK": 0.4550, "HKD": 1.3536, "HUF": 0.0289, "IDR": 0.0007, "ILS": 2.8540,
    "INR": 0.1262, "ISK": 0.0766, "KRW": 0.0078, "MXN": 0.5796, "MYR": 2.3127,
    "NZD": 6.3946, "PHP": 0.1844, "PLN": 2.6551, "RON": 2.2981, "SGD": 7.9076,
    "THB": 0.2997, "TRY": 0.3220
}

df_transactions["amount_sek"] = df_transactions.apply(
    lambda row: round(row["amount"] * conversion_rates.get(row["currency"], 0), 2),
    axis=1
)

logga("Currency convertions done.")


[2025-06-04 17:24:17] Starting currency converter to SEK...
[2025-06-04 17:24:18] Currency convertions done.


In [4]:
logga("Extracting and controlling data...")

print("Columns in df_transactions:", df_transactions.columns.tolist())

null_check = df_transactions.drop(columns=["notes"]).isnull().any(axis=1)
invalid_transactions = df_transactions[null_check]
logga(f"Invalid columns (except for notes): {len(invalid_transactions)}")

missing_notes = df_transactions["notes"].isnull() | (df_transactions["notes"].str.strip() == "")
logga(f"Invalid notes: {missing_notes.sum()}")


[2025-06-04 17:00:22] Extracting and controlling data...
Columns in df_transactions: ['transaction_id', 'timestamp', 'amount', 'currency', 'sender_account', 'receiver_account', 'sender_country', 'sender_municipality', 'receiver_country', 'receiver_municipality', 'transaction_type', 'notes', 'amount_sek']
[2025-06-04 17:00:22] Invalid columns (except for notes): 1990
[2025-06-04 17:00:22] Invalid notes: 9948


In [5]:
logga("Extracting nightly transactions (00:00–05:59)...")

df_transactions["hour"] = df_transactions["timestamp"].dt.hour
night_tx = df_transactions[(df_transactions["hour"] >= 0) & (df_transactions["hour"] <= 5)]

logga(f"Nighlt transactions: {len(night_tx)}")
night_tx[["transaction_id", "timestamp", "amount", "currency"]].head()


[2025-06-04 17:01:45] Extracting nightly transactions (00:00–05:59)...
[2025-06-04 17:01:45] Nighlt transactions: 23576


,transaction_id,timestamp,amount,currency
0,62cacc89-95ca-41c0-a06f-d12950c85a37,2025-01-08 03:17:00,1174.47,SEK
11,0308a664-8b78-40bd-9281-8623b3d2584a,2025-03-08 00:00:00,10977.56,SEK
14,ccc76011-9857-40a8-b951-b8c90144449a,2025-04-09 05:56:00,31999.11,SEK
16,e0827340-f916-4f38-b268-ed6570416886,2025-02-26 04:23:00,15452.26,SEK
17,4fab6c87-e40f-4e77-8dc2-43aa086c946c,2025-03-20 01:11:00,23696.98,SEK


In [5]:
logga("Extracting potential smurfing transactions (<1000 SEK)...")

small_tx = df_transactions[df_transactions["amount"] < 1000]
logga(f"Small transactions: {len(small_tx)}")

senders = small_tx.groupby("receiver_account")["sender_account"].nunique().reset_index()
smurf_candidates = senders[senders["sender_account"] > 5]
logga(f"Recievers with >5 unique senders: {len(smurf_candidates)}")

smurf_tx = small_tx[small_tx["receiver_account"].isin(smurf_candidates["receiver_account"])]
logga(f"Potential smurfing transactions identified: {len(smurf_tx)}")

smurf_tx.head()


[2025-06-04 17:24:22] Extracting potential smurfing transactions (<1000 SEK)...
[2025-06-04 17:24:22] Small transactions: 7077
[2025-06-04 17:24:22] Recievers with >5 unique senders: 32
[2025-06-04 17:24:22] Potential smurfing transactions identified: 345


,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes,amount_sek
1156,0768d874-012e-4e3d-ab57-d7e7abd68613,2025-02-28 11:26:00,628.56,SEK,SE8902ZWKL91581622045409,SE8902ZNHI21783939608864,Sweden,Östersund,Sweden,Ängelholm,incoming,NaN,0.0
1698,fe64bd92-b0dc-4b0f-a112-cf9e6ca89824,2025-02-04 01:48:00,154.70,SEK,SE8902NDNE36671961231780,SE8902XXAF74151570423565,Sweden,Linköping,Sweden,Skellefteå,incoming,Payment for invoice #1833,0.0
1958,8b4a5e0c-274c-43b0-a611-e973fbef2b1c,2025-03-14 07:15:00,939.61,SEK,SE8902FCSV26977507325534,SE8902MQZJ06643246897705,Sweden,Falun,Sweden,Motala,outgoing,Loan repayment,0.0
4606,40e6acbd-d419-42a4-beaa-e5424b7b697e,2025-04-02 12:28:00,149.23,SEK,SE8902UGGW38274485777478,SE8902MQZJ06643246897705,Sweden,Växjö,Sweden,Skellefteå,incoming,Payment for invoice #1096,0.0
4881,8c26b18f-d9c9-4d57-97c9-bfb68adce2dd,2025-05-06 13:04:00,550.96,SEK,SE8902ZJEO82962448424927,SE8902XXAF74151570423565,Sweden,Malmö,Sweden,Karlskrona,outgoing,Utility bill,0.0


In [10]:
logga("Extracting accounts with >5 transactions in 24h...")

df_transactions["date"] = df_transactions["timestamp"].dt.date

daily_transactions_counts = df_transactions.groupby(["sender_account", "date"]).size().reset_index(name="transactions_count")
frequent_daily_senders = daily_transactions_counts[daily_transactions_counts["transactions_count"] > 5]

logga(f"Accounts with >5 transactions in 24h: {len(frequent_daily_senders)}")
frequent_daily_senders.head(10)

[2025-06-04 17:31:51] Extracting accounts with >5 transactions in 24h...
[2025-06-04 17:31:51] Accounts with >5 transactions in 24h: 50


,sender_account,date,transactions_count
43772,SE8902ATDH36917846037146,2025-01-13,94
44260,SE8902BABM26306156053804,2025-02-02,99
45399,SE8902BXJC93392072105707,2025-02-15,99
45848,SE8902CFVV30512683866714,2025-05-06,100
46229,SE8902CODF26383416794081,2025-03-04,99
46518,SE8902CUNN22125220865735,2025-02-15,97
47299,SE8902DIGS61927513105237,2025-03-03,100
47356,SE8902DJPX89629848207656,2025-04-05,97
48217,SE8902DYMK28727836359365,2025-04-04,99
49459,SE8902EPZK82020056741347,2025-03-14,98


In [11]:
logga("Extracting self transfers...")

same_account = df_transactions[df_transactions["sender_account"] == df_transactions["receiver_account"]]

logga(f"Self transfers: {len(same_account)} st")
same_account.head(10)


[2025-06-04 17:33:10] Extracting self transfers...
[2025-06-04 17:33:11] Self transfers: 12 st


,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes,amount_sek,currency_risk,date
326,5c3baa6f-289f-481c-95e8-18957ce653a2,2025-04-20 23:07:00,15614.07,SEK,SE8902KWRS27148391651848,SE8902KWRS27148391651848,Sweden,Uppsala,Sweden,Lidköping,incoming,Freelance project payment,0.0,Low,2025-04-20
627,7ee1c800-65d3-4d86-8077-07c754d8464c,2025-04-29 16:18:00,23421.03,SEK,SE8902FBTR73223116722714,SE8902FBTR73223116722714,Sweden,Ängelholm,Sweden,Kalmar,incoming,Reimbursement for travel,0.0,Low,2025-04-29
1139,33f27841-93b7-41dc-aec1-ff796d18fd6e,2025-05-02 00:09:00,21287.34,SEK,SE8902GUBN02340962282734,SE8902GUBN02340962282734,Sweden,Nyköping,Sweden,Lidköping,incoming,Gift to family member,0.0,Low,2025-05-02
3428,83167861-8440-4f9d-a6c7-d5dc24170d8f,2025-05-01 23:08:00,47855.61,SEK,SE8902HCFQ50732586147290,SE8902HCFQ50732586147290,Sweden,Umeå,Sweden,Uddevalla,outgoing,Gift to family member,0.0,Low,2025-05-01
3471,4f9d8e01-80f8-4a00-966b-cdc8397bddd8,2025-01-01 15:15:00,32433.22,SEK,SE8902TJBO76009734317061,SE8902TJBO76009734317061,Sweden,Helsingborg,Sweden,Trelleborg,outgoing,Subscription fee,0.0,Low,2025-01-01
3660,c75aae3b-fd11-409c-9961-b9f1cf095bd0,2025-03-22 10:24:00,42640.40,SEK,SE8902JHMA69643058655073,SE8902JHMA69643058655073,Sweden,Västerås,Sweden,Eskilstuna,incoming,Subscription fee,0.0,Low,2025-03-22
4410,ec6d64bf-7f17-490e-9be7-9cf0bedd7733,2025-02-22 03:12:00,37328.08,SEK,SE8902KWRS27148391651848,SE8902KWRS27148391651848,Sweden,Falun,Sweden,Sundsvall,incoming,Freelance project payment,0.0,Low,2025-02-22
6526,8a2affc9-a2ee-4f84-af04-cbb726f5fa52,2025-04-18 03:15:00,33174.44,SEK,SE8902KWAF90270853352878,SE8902KWAF90270853352878,Sweden,Kalmar,Sweden,Skövde,outgoing,Consulting fee,0.0,Low,2025-04-18
8399,438b3d21-008e-4644-9714-f6fcc46648dd,2025-01-04 00:13:00,23491.08,SEK,SE8902IAAP17064707009052,SE8902IAAP17064707009052,Sweden,Gävle,Sweden,Eskilstuna,outgoing,Utility bill,0.0,Low,2025-01-04
8751,cd104bad-80f1-45c9-9562-45e46340ca84,2025-03-19 23:24:00,11035.39,SEK,SE8902NRAS31362495279847,SE8902NRAS31362495279847,Sweden,Landskrona,Sweden,Piteå,outgoing,Rent transfer,0.0,Low,2025-03-19


In [13]:
logga("Extracting internal transactions (same account holder but different bank account)...")

# Koppla konton till personnummer
account_to_pnr = df_customers.set_index("BankAccount")["Personnummer"].to_dict()
df_transactions["sender_pnr"] = df_transactions["sender_account"].map(account_to_pnr)
df_transactions["receiver_pnr"] = df_transactions["receiver_account"].map(account_to_pnr)

# Filtrera interna överföringar
internal_same_person = df_transactions[
    (df_transactions["sender_account"] != df_transactions["receiver_account"]) &
    (df_transactions["sender_pnr"].notnull()) &
    (df_transactions["sender_pnr"] == df_transactions["receiver_pnr"])
]

logga(f"Internal transactions (same account holder but different bank account): {len(internal_same_person)}")

internal_same_person[[
    "transaction_id", "sender_account", "receiver_account",
    "sender_pnr", "receiver_pnr", "amount"
]].head(10)


[2025-06-04 17:36:07] Extracting internal transactions (same account holder but different bank account)...
[2025-06-04 17:36:07] Internal transactions (same account holder but different bank account): 11


,transaction_id,sender_account,receiver_account,sender_pnr,receiver_pnr,amount
784,833c2411-18a9-48a2-a4d4-e48afb1b2dd9,SE8902UNAU13611272599268,SE8902YHDH55973704252760,390120-3251,390120-3251,6197.94
913,4a4c682c-c339-4e43-9dbb-b8651df679fa,SE8902HXLB10149156679316,SE8902PCFY59781759139715,441022-8888,441022-8888,47490.83
1246,9fbed290-ad2a-4831-bc4a-cdd72b134a20,SE8902QCNP19721394374879,SE8902XUOQ37054183061149,891030-3471,891030-3471,15693.57
1371,5ed8c931-eba1-4faa-902d-3d9991ac6df3,SE8902WQME16820619653293,SE8902KSGB25510595835295,060925-9098,060925-9098,34514.08
4833,04957ffa-90a3-4e82-8628-b1c9520fcf64,SE8902RASX95864855242605,SE8902OCQK11725418237602,850630-1954,850630-1954,2095.26
5340,84f7cc35-8b38-4828-9f5f-1928a08aa577,SE8902UDAI89693707440786,SE8902PEJL40222448009128,570120-3019,570120-3019,25702.11
5427,cc98e7b9-0539-475c-b80b-29530418881d,SE8902RBXU00091422402190,SE8902LFCU74107203362240,911111-4402,911111-4402,43462.36
5949,c92761ab-d06b-4766-b527-cc84e8126e08,SE8902IAAP17064707009052,SE8902PMAG08583189181534,600315-9438,600315-9438,28096.18
7518,5b0b03ba-5cde-495b-a07a-1740d4d9b6ef,SE8902GORN68994003453353,SE8902FCSV26977507325534,780805-2190,780805-2190,40974.98
7816,e18cb05e-4cc8-4985-8214-936ff140827d,SE8902TKRB93799625518098,SE8902IRXN70281866292484,990131-4873,990131-4873,35235.07


In [12]:
logga("Extracting international transfers...")

non_swedish = df_transactions[
    (df_transactions["sender_country"].str.lower() != "sweden") |
    (df_transactions["receiver_country"].str.lower() != "sweden")
]

logga(f"International transactions: {len(non_swedish)}")
non_swedish.head(10)


[2025-06-04 17:34:12] Extracting international transfers...
[2025-06-04 17:34:12] International transactions: 43723


,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes,amount_sek,currency_risk,date
11,0308a664-8b78-40bd-9281-8623b3d2584a,2025-03-08 00:00:00,10977.56,SEK,SE8902MPBO87772633723937,SE8902ZJVA11817265425330,NaN,Uddevalla,Sweden,Skövde,outgoing,Rent transfer,0.0,Low,2025-03-08
43,32b8ef63-4f95-4891-a39a-8b67a2ca57d9,2025-05-02 06:25:00,30955.77,SEK,SE8902NMLM36956633220058,SE8902BWKM27720380863718,NaN,Örnsköldsvik,Sweden,Gävle,incoming,Freelance project payment,0.0,Low,2025-05-02
520,ccd9d591-1e7e-4924-9565-406698971238,2025-04-28 09:34:00,15551.69,SEK,SE8902IWLJ80369151603079,SE8902QVDX07065136908695,NaN,Östersund,Sweden,Sandviken,outgoing,NaN,0.0,Low,2025-04-28
675,99723967-1bc3-4f9f-85ed-cc2bfe675f5c,2025-04-17 11:12:00,46995.74,SEK,SE8902JWPZ44810854846338,SE8902PVDU32397262267370,Sweden,Nyköping,NaN,Falun,outgoing,Salary payment,0.0,Low,2025-04-17
742,1dd877ea-185c-4f6c-b3f4-668032000d9e,2025-05-12 23:53:00,49040.75,SEK,SE8902JFXZ40334167631989,SE8902DGRO18476288845448,Sweden,Östersund,NaN,Uddevalla,outgoing,Insurance payout,0.0,Low,2025-05-12
762,39794385-3319-4f1d-b73a-5830638aa2a6,2025-04-10 23:55:00,15630.05,SEK,SE8902OLFT37200388827050,SE8902FZSN55947389902845,Sweden,Södertälje,NaN,Lidköping,incoming,Loan repayment,0.0,Low,2025-04-10
844,9ec868ed-f7bf-4e02-8133-6010462c4d44,2025-03-18 04:11:00,32780.29,SEK,SE8902IIVG09252145366202,SE8902ORRS48118816092087,NaN,Stockholm,Sweden,Kristianstad,outgoing,Gift to family member,0.0,Low,2025-03-18
845,5b2636bb-70eb-4204-bc42-1b5bc87f3e6e,2025-05-12 03:55:00,29449.80,SEK,SE8902GZKC59110904284187,SE8902CFEY43701881421531,Sweden,Umeå,NaN,Mölndal,outgoing,NaN,0.0,Low,2025-05-12
906,b2daea58-16d5-40ae-ba90-3d844362b519,2025-03-14 14:17:00,43954.08,SEK,SE8902UFZY41218181108055,SE8902XBIN60287954526260,Sweden,Kalmar,NaN,Sandviken,outgoing,Insurance payout,0.0,Low,2025-03-14
999,746fe0d9-03f2-4b36-a00f-66a3fe73371f,2025-04-30 07:58:00,31230.62,SEK,SE8902MKFC17006139746737,SE8902LHUA50734331657544,NaN,Örnsköldsvik,Sweden,Skövde,incoming,Loan repayment,0.0,Low,2025-04-30


In [7]:
logga("Starting risk classifications for currencies...")

low_risk_currencies = {"SEK", "JPY"}
moderate_risk_currencies = {
    "EUR", "GBP", "USD", "NOK", "DKK", "CHF", "SGD", "HKD", "AUD",
    "NZD", "CAD", "HUF", "CZK", "RON", "PLN", "BGN", "KRW", "ISK", "MYR", "ILS"
}
high_risk_currencies = {
    "ZMW", "ZAR", "TRY", "BRL", "IDR", "MXN", "PHP", "INR", "THB", "CNY", "RMB"
}

def classify_currency_risk(currency):
    c = str(currency).upper()
    if c in high_risk_currencies:
        return "High"
    elif c in moderate_risk_currencies:
        return "Moderate"
    elif c in low_risk_currencies:
        return "Low"
    else:
        return "Unknown"

df_transactions["currency_risk"] = df_transactions["currency"].apply(classify_currency_risk)
logga("Risk classification done. Creating summary...")

risk_summary = df_transactions["currency_risk"].value_counts(normalize=False).reset_index()
risk_summary.columns = ["Risk level", "Number of transactions"]
risk_summary["Percent"] = round((risk_summary["Number of transactions"] / len(df_transactions)) * 100, 2)

for _, rad in risk_summary.iterrows():
    logga(f"{rad['Risk level']} risk: {rad['Number of transactions']} st ({rad['Percent']}%)")

risk_summary.head()


[2025-06-04 17:25:57] Starting risk classifications for currencies...
[2025-06-04 17:25:57] Risk classification done. Creating summary...
[2025-06-04 17:25:57] Low risk: 80776 st (80.78%)
[2025-06-04 17:25:57] Moderate risk: 18984 st (18.98%)
[2025-06-04 17:25:57] High risk: 240 st (0.24%)


,Risk level,Number of transactions,Percent
0,Low,80776,80.78
1,Moderate,18984,18.98
2,High,240,0.24


In [8]:
high_risk_currency_transactions = df_transactions[df_transactions["currency_risk"] == "High"]
logga(f"High risk transactions: {len(high_risk_currency_transactions)}")

high_risk_currency_transactions[[
    "transaction_id", "timestamp", "currency", "amount", "amount_sek",
    "sender_account", "receiver_account", "currency_risk"
]].head(10)


[2025-06-04 17:26:36] High risk transactions: 240


,transaction_id,timestamp,currency,amount,amount_sek,sender_account,receiver_account,currency_risk
10239,f05eaab6-7a98-4d46-a020-45e8e1409e58,2025-04-15 16:57:00,ZMW,41839.42,14995.25,GB42QTTR07621951221751,SE8902FOBI10005828618756,High
11319,0511f0f8-58db-4d7c-aba9-587fddf3c67c,2025-05-17 16:03:00,ZAR,20652.99,11962.21,GB38JUGJ63276092779398,SE8902XWUV40935991476872,High
11584,dcf17045-a4d4-430c-8c5b-fd9cdcdfa27d,2025-01-08 20:37:00,ZMW,17248.16,6181.74,GB80NWUS06322321309531,SE8902YBUM91649266178467,High
11711,329ccd73-eefa-44aa-bbe5-50bb7b300398,2025-03-19 17:25:00,ZMW,25272.64,9057.71,GB68LJCV31691149334671,SE8902VWKO20169767635100,High
11814,370ebdf7-8482-4190-b1fe-e16cd5d64de7,2025-01-05 05:16:00,ZMW,28192.47,10104.18,GB52PXOH81059438551311,SE8902VJUZ52218371196993,High
12101,9fca9229-e1b2-4db4-be0a-6fb18fe91910,2025-05-04 04:40:00,RMB,24662.12,35434.53,GB21RGSF26047710313683,SE8902OVSC20334070794606,High
12368,13644261-095f-48dd-b4de-e6bac17a8d3e,2025-04-15 14:12:00,RMB,27123.17,38970.57,GB40SARN58167009029188,SE8902QLUT99208994499946,High
12795,8add0efd-e516-4eb4-be3c-b1fb9c8513c5,2025-03-15 10:18:00,RMB,34969.10,50243.60,GB81WPUB16516308301598,SE8902XVJQ62588665096343,High
12831,23c6c191-26f4-4c88-af85-a323063c7431,2025-03-02 00:45:00,ZMW,44544.90,15964.89,GB86FVEX16733318116301,SE8902OSQP10294973862487,High
13581,8e447617-09f2-4077-a08b-ff0e1c83b394,2025-01-02 22:56:00,ZAR,19725.48,11425.00,GB69VDBV44938264975661,SE8902NQVT00867426002134,High


In [9]:
moderate_risk_currency_transactions = df_transactions[df_transactions["currency_risk"] == "Moderate"]
logga(f"Moderate risk transactions: {len(moderate_risk_currency_transactions)}")

moderate_risk_currency_transactions[[
    "transaction_id", "currency", "amount", "currency_risk"
]].head(10)


[2025-06-04 17:27:04] Moderate risk transactions: 18984


,transaction_id,currency,amount,currency_risk
10005,9cc03d06-a90f-4088-a455-a1206e4a113b,DKK,43050.16,Moderate
10008,2b5ea179-a55e-472b-a674-dbedd825b264,USD,18985.04,Moderate
10011,6b1dd651-c8c4-4c8e-bb50-0e9f79c55951,USD,32978.19,Moderate
10015,6db71aee-b499-43b9-b2bd-0d84902c37fd,EUR,21305.93,Moderate
10020,d66a4cf1-ec71-4cb9-82ba-7f1504052e15,EUR,39833.97,Moderate
10031,75878fbe-70f7-490d-96e8-3b31160f423e,USD,33218.45,Moderate
10037,9cbfac60-0fb6-4f74-9b01-2fe1f4f21aaa,EUR,23318.82,Moderate
10045,6dcbe044-c58e-4d19-8e9b-8f85f7d826b4,USD,15820.70,Moderate
10052,50d6c8ca-b52a-4638-9ba5-8992b6e08440,USD,34490.24,Moderate
10054,96e7ceda-f468-404c-9585-677416a8eef7,USD,27737.07,Moderate
